# Practical session 3 - Brownian motion, Fourier transform

Students (pair):
- [Student 1]([link](https://github.com/username1))
- [Student 2]([link](https://github.com/username2))

In [1]:
%load_ext autoreload
%autoreload 2

## <a name="ex1">Exercise 1: Brownian motion</a>

This first exercise consists in generating a Brownian motion on the closed unit ball $\mathcal{B}(\mathbf{0}, 1) = \{ \mathbf{x} \mid \Vert \mathbf{x} \Vert  \leq 1\}$, focusing first on the 2-D case. The Brownian motion is a random walk with independent, identically distributed Gaussian increments, appearing for instance in thermodynamics and statistical mechanics (to model the evolution of a large particle in a medium composed of a large number of small particles, ...). It is also connected to the diffusion process (Einstein).

Let $N \in \mathbb{N}^*$, $\delta > 0$, and $\mathbf{x} = (x_1, x_2) \in  \mathcal{B}(\mathbf{0}, 1)$. The first $N$ steps of a 2-D discrete-time Brownian motion $W$ can be generated as follows

\begin{align*}
    W_0 &= \mathbf{x}, \\
    %
    (\forall n \in \{1, \dotsc, N-1 \}), \quad W_n &= W_{n−1} + \sqrt{\delta} G_n, \quad G_n \sim \mathcal{N}(\mathbf{0}, \mathbf{I}),
\end{align*}

where $\mathcal{N}(\mathbf{0}, \mathbf{I})$ is a Gaussian distribution with mean $\mathbf{0}$ and identity covariance matrix.

1. Define a random generator `rng`, set to a known state for reproducibility (see session 2).

**Answer:**

In [2]:
import random as rd
import numpy as np

rd.seed(42)

from typing import Optional

def rng(n: Optional[int]) -> np.ndarray:
    return np.random.multivariate_normal(np.array([0,0]), np.identity(2), n)
#If n=None, return a simple random 2D point
#If n is an integer, return an array of n random 2D points

2. Implement a function `brownian_motion(niter, x, step, rng)` which

    - simulates $W$ until it reaches the boundary of $\mathcal{B}(\mathbf{0}, 1)$, using a maximum of $N$ iterations (`niter`), a starting point $\mathbf{x} \in \mathcal{B}(\mathbf{0}, 1)$ (`x`) and step-size $\delta$ (`step`);
    - interpolates linearly between the two last positions to determine the points $W^*$ where the trajectory crosses the boundary (if applicable);
    - returns both the whole random walk $W$ and, if appropriate, the point at the intersection between the last segment of the trajectory and $\mathcal{B}(\mathbf{0}, 1)$.
 
> Hint: 
> - you can easily derive a closed form expression for $W^*$, observing that $\Vert W^* \Vert^2= 1$ and $W^* \in [W_{n-1}, W_n]$. 
> - you can also take a look at [`np.roots`](https://numpy.org/doc/stable/reference/generated/numpy.roots.html?highlight=roots#numpy.roots) if needed.

**Answer:**

In [41]:
def norm(x):
    return np.linalg.norm(x)

def finding_intersection(x1, x2):
    m=0.5*x1+0.5*x2
    start=x1
    end=x2
    while np.abs((norm(m)-1))>1e-18:
        if norm(m)>1:
            end=m
        else:
            start=m
        m=0.5*start+0.5*end
    return m

def linear_fct(x1, x2):
    print(x2-x1)
    a = (x2[1]-x1[1])/(x2[0]-x1[0])
    b = x1[1]-a*x1[0]
    print(a,b)
    return np.array([a, b])

def intersect_circle(lin):
    slope, origin = lin
    print(slope, origin)
    a, b, c = slope**2-1, 2*slope*origin, origin**2-1
    print(a,b,c)
    return np.array([a,b,c])

def approx(x1, x2):
    x= x2[0]-x1[0]
    y= x2[1]-x1[1]
    a=x**2+y**2
    b=-2*y**2
    c=y**2
    return np.roots([a,b,c])

def brownian_motion(niter,x,step,rng) -> np.ndarray:
    """ Simulate a Brownian motion with niter steps, starting from x,
        with increments of size step, using the random number generator rng.
    """
    X=x
    traj=[X]
    iter=0
    while (iter<niter and norm(X)<1):
        X=X+np.sqrt(step)*rng(None)
        traj.append(X)
        iter+=1
    if iter<niter:
        wstar = finding_intersection(traj[-2], traj[-1])
        traj[-1]=wstar
    else:
        wstar=None
    # x_star = np.roots(intersect_circle(linear_fct(traj[-2], traj[-1])))
    # y_star = np.sqrt(1 - x_star[0]**2)
    return traj

brownian_motion(1000, np.array([0.2, 0.4]), 0.01, rng)


[array([0.2, 0.4]),
 array([0.33924392, 0.39454593]),
 array([0.42484768, 0.42396968]),
 array([0.47304305, 0.35148503]),
 array([0.54237937, 0.43483332]),
 array([0.65061281, 0.29467568]),
 array([0.69642896, 0.10840797]),
 array([0.60214268, 0.11255076]),
 array([0.59533777, 0.18371825]),
 array([0.52444798, 0.22318096]),
 array([0.42136735, 0.25603653]),
 array([0.30102959, 0.35333008]),
 array([0.19911711, 0.49090913]),
 array([0.17215451, 0.50836298]),
 array([0.14679254, 0.22634535]),
 array([0.18827204, 0.07302721]),
 array([ 0.25175734, -0.00518348]),
 array([0.29067425, 0.01208743]),
 array([0.26592094, 0.0566095 ]),
 array([0.3801186 , 0.10393805]),
 array([0.32539427, 0.10257444]),
 array([0.26585433, 0.31144844]),
 array([0.26361306, 0.41926516]),
 array([0.12680501, 0.28912085]),
 array([0.18039924, 0.26866856]),
 array([0.16476457, 0.0867871 ]),
 array([0.20823766, 0.07943224]),
 array([0.25254424, 0.02397075]),
 array([0.26218586, 0.09637426]),
 array([0.14466042, 0.0618

In [4]:
np.linalg.norm(np.array([0.48688796, 0.87862456]))

1.0045103299758322

3. Diplay the trajectory of a Brownian motion starting from $\mathbf{x} = (0.2, 0.4)$, using $\delta = 10^{-2}$, $N = 1000$. Display the unit circle on the same figure, and highlight the intersection with the boundary of the domain (whenever it exists).

> Hint: to draw the unit disk, you can use for instance:
> ```python
> circle = plt.Circle((0,0), 1)
> fig, ax = plt.subplots()
> plt.xlim(-1.25,1.25)
> plt.ylim(-1.25,1.25)
> plt.grid(linestyle = "--", zorder = 1)
> ax.set_aspect(1)
> ax.add_artist(circle)
> ```

**Answer:**

In [5]:
# your code

4. Represent, on the same figure, 4 other trajectories of $W$ with the same parameters.

**Answer:**

In [6]:
# your code

5. [Bonus] Generalize the procedure to a $M$-dimensional Brownian motion, $M > 2$.

**Answer:**

In [7]:
# your code

---
## <a name="ex2">Exercise 2: 2D Fourier transform, ideal low-pass filter and linear convolution</a>

In this exercise, we explore the use of the 2-dimensional Fourier transform to filter an image, and convolve it with a blurring kernel.

1\. Load and display one of the images contained in the `img/` folder. The image will be denoted by $\mathbf{X} \in \mathbb{R}^{M_1 \times N_1}$ in the rest of this exercise.

**Answer:**

In [8]:
# your code

2\. Let $\mathcal{F}$ denote the 2D discrete Fourier transform. Compute $|\mathcal{F}(\mathbf{X})|^2$, the spectrum of the image $\mathbf{X} \in \mathbb{R}^{M_1 \times N_1}$ (i.e., the term-wise squared absolute value of its Fourier transform) loaded in 1. Display the result in logarithmic scale.

a) In this representation, where is the pixel of the spectrum associated with the null frequency located?
    
b) Take a look at the documentation of `np.fft.fftshift`. Use it to ensure that the null frequency is located at the center of the image. 

**Answer:**

In [9]:
# your code

3\. 
    a) Create a function `ideal_lowpass_filter` to filter $\mathbf{X}$ by an ideal low-pass filter. The filter preserves Fourier coefficients associated to frequencies below a cutoff specified in each direction ($\mathbf{f}_c = (f_{c,y}, f_{c,x})$), and sets others to zero. For simplicity, $f_{c,y}$ and $f_{c,x}$ can be expressed as a number of samples to be kept along each dimension (e.g., $\mathbf{f}_c = (50,50)$).

b) Display the filtered image for 2 different values of $\mathbf{f}_c$. What do you observe as the cutoff frequencies increase?
    
> Warning: beware the type of the array after `np.fft.fft2`, do not hesitate to specify the type if you make copies from this array
> ```python
> a = np.zeros((2,2), dtype=np.complex)
> ...
> ```

**Answer:**

In [10]:
# your code

4\. Let $\mathbf{H} \in \mathbb{R}^{M_2\times N_2}$ be a 2-D Gaussian kernel, obtained as the outer product of two 1-D Gaussian windows $\mathbf{w}_y \in \mathbb{R}^{M_2}$ and $\mathbf{w}_x \in \mathbb{R}^{N_2}$, of standard deviation $\sigma_y = 10$ and $\sigma_x = 10$, respectively:

\begin{equation}
    \mathbf{H} = \mathbf{w}_y \mathbf{w}_x^T.
\end{equation}

Let $M = M_1+M_2-1$ and $N =  N_1+N_2-1$. From the discrete convolution theorem, the linear convolution between $\mathbf{H}$ and $\mathbf{X}$ can be computed as follows

\begin{equation}
    \mathbf{X} \star \mathbf{H} = \mathcal{F}^{-1} \Big( \mathcal{F}\big(P_1(\mathbf{X})\big) \odot \mathcal{F}\big(P_2(\mathbf{H})\big) \Big) \in \mathbb{R}^{M\times N},
\end{equation}

where $P_i: \mathbb{R}^{M_i \times N_i} \rightarrow \mathbb{R}^{M \times N}$, $i \in \{1, 2\}$, are 0-padding operators, $\odot$ is the Hadamard (= term-wise) product, $\mathcal{F}^{-1}$ is the 2D discrete inverse Fourier transform.

Compute and display $\mathbf{X} \star \mathbf{H}$, for $M_2 = N_2 = 10$. What do you observe?

> Hint: 
> - the usual 0-padding procedure in image space consists in appending trailing zeros. For instance (in 1D), 0-padding a vector $\mathbf{x} \in \mathbb{R}^N_1$ to the size $N>N_1$ corresponds to creating the vector
\begin{bmatrix}
\mathbf{x} \\
\mathbf{0}_{N-N_1}
\end{bmatrix}
> - since the input images are real, $\mathcal{F}(\mathbf{x})$ and $\mathcal{F}(\mathbf{h})$ are Hermitian symmetric. In this case, a more efficient version of `np.fft.fft2` can be used, computing only quarter of the Fourier coefficients (half of the Fourier coefficients in each direction): [`np.fft.rfft2`](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft2.html?highlight=rfft#numpy.fft.rfft2). Its inverse, [`np.fft.irfft2`](https://numpy.org/doc/stable/reference/generated/numpy.fft.irfft2.html#numpy.fft.irfft2), also ensures that the output is real;
> - the 2D Gaussian window can be generated as the outer product of two 1D Gaussian windows (one window for each dimension);
> - you can take a look at [scipy.signal.windows.gaussian](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.gaussian.html#scipy.signal.windows.gaussian) and [np.newaxis](https://numpy.org/doc/stable/reference/constants.html?highlight=newaxis#numpy.newaxis).

**Answer:**

In [11]:
# your code